# Wind max-in-hour experiments figures (fig 3)

In this notebook we:
1. evaluate the wind max-in-hour hindcast experiment and generate figure 3, 
2. produce a station maps plot, and
3. calculate if the difference in errors is statistically significant.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

from scores.continuous import mse
from scores.processing import broadcast_and_match_nan
from scores.stats.statistical_tests import diebold_mariano

import plotly.express as px
import plotly.graph_objects as go

### Get Data for wind max-in-hour experiment

In [ ]:
official_max_exp = xr.open_dataarray(
    "data/windmax_exp/Official_WindMag_00_20190901-20191130.nc"
)
# Note that the 12Z AutoFcst was the automated guidance available to meteorologists for the
# afternoon (00Z) official forecast issue.
autofcst_max_exp = xr.open_dataarray(
    "data/windmax_exp/FCF_2_0_AutoFcst_WindMag_18_20190901-20191130.nc"
)
hindcast_max_exp = xr.open_dataarray(
    "data/windmax_exp/AutoFcstMax_WindMag_12_20190901-20191130.nc"
)
obs_max_exp = xr.open_dataarray(
    "data/windmax_exp/obs_WindMagMaxInHour_20190901-20191130.nc"
)

# Match missing data between datasets
(
    official_max_exp,
    autofcst_max_exp,
    hindcast_max_exp,
    obs_max_exp,
) = broadcast_and_match_nan(
    official_max_exp, autofcst_max_exp, hindcast_max_exp, obs_max_exp
)

### Calulate MSE

In [ ]:
official_max_exp_mse = mse(official_max_exp, obs_max_exp, preserve_dims="lead_day")
hindcast_max_exp_mse = mse(hindcast_max_exp, obs_max_exp, preserve_dims="lead_day")
autofcst_max_exp_mse = mse(autofcst_max_exp, obs_max_exp, preserve_dims="lead_day")

In [ ]:
official_line_colour = "rgba(230,159,0,1)"
autofcst_line_colour = "rgba(86,180,233,1)"
hindcast_line_colour = "#009E73"

# Left subfigure. WindMax experiment
figure = go.Figure()
figure.add_trace(
    go.Scatter(
        x=official_max_exp_mse.lead_day,
        y=official_max_exp_mse.values,
        line=dict(color=official_line_colour),
        name="Official",
    )
)
figure.add_trace(
    go.Scatter(
        x=autofcst_max_exp_mse.lead_day,
        y=autofcst_max_exp_mse.values,
        line=dict(color=autofcst_line_colour),
        name="Existing automated",
    )
)
figure.add_trace(
    go.Scatter(
        x=hindcast_max_exp_mse.lead_day,
        y=hindcast_max_exp_mse.values,
        line=dict(color=hindcast_line_colour),
        name="Hindcast experiment",
    )
)
figure.update_layout(
    legend=dict(x=0.01, y=0.99),
    height=400,
    width=400,
    margin=go.layout.Margin(
        l=20,  # left margin
        r=20,  # right margin
        b=20,  # bottom margin
        t=20,  # top margin
    ),
)
figure.update_yaxes(title_text="MSE (kt<sup>2</sup>)")
figure.update_xaxes(title_text="Lead day", tickmode="linear", tick0=0, dtick=1)

In [ ]:
figure.write_image("results/figures/wind_max_exp.pdf")

### Station map

In [ ]:
df = pd.read_csv("data/aws_metadata/station_data.csv")
df = df[df["station_number"].isin(official_max_exp.station_number.values)]


fig = px.scatter_geo(
    df, lat="LATITUDE", lon="LONGITUDE", color_discrete_sequence=["red"]
)

fig.update_geos(
    resolution=50,
    lonaxis_range=[110, 155],
    lataxis_range=[-45, -10],
    showcoastlines=True,
    showland=True,
    showocean=True,
    oceancolor="rgb(144, 195, 245)",
    showcountries=True,
    showframe=True,
    lonaxis=dict(showgrid=True, gridcolor="gray", gridwidth=0.5, dtick=5),
    lataxis=dict(showgrid=True, gridcolor="gray", gridwidth=0.5, dtick=5),
)

fig.update_traces(marker={"size": 4})
fig.update_layout(
    height=350,
    width=400,
    margin=go.layout.Margin(
        l=0,  # left margin
        r=0,  # right margin
        b=0,  # bottom margin
        t=0,  # top margin
    ),
)
fig.show()

In [ ]:
fig.write_image("results/station_maps/a_wind_aus_stations.pdf")

### Statistical significance testing for wind max experiment

In [ ]:
official_max_exp_mse = mse(
    official_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"]
)
hindcast_max_exp_mse = mse(
    hindcast_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"]
)
autofcst_max_exp_mse = mse(
    autofcst_max_exp, obs_max_exp, preserve_dims=["lead_day", "valid_start"]
)

In [ ]:
# Difference between Official and the hindcast
diff_official_hindcast = official_max_exp_mse - hindcast_max_exp_mse
diff_official_hindcast = diff_official_hindcast.assign_coords(
    h=("lead_day", [2, 3, 4, 5, 6, 7, 8])
)
dm_result = diebold_mariano(diff_official_hindcast, "lead_day", "h")
dm_result

In [ ]:
# Difference between the existing AutoFcst and the hindcast
diff_autofcst_hindcast = autofcst_max_exp_mse - hindcast_max_exp_mse
diff_autofcst_hindcast = diff_autofcst_hindcast.assign_coords(
    h=("lead_day", [2, 3, 4, 5, 6, 7, 8])
)
dm_result = diebold_mariano(diff_autofcst_hindcast, "lead_day", "h")
dm_result

In [ ]:
# Difference between the existing AutoFcst and the Official
diff_autofcst_official = autofcst_max_exp_mse - official_max_exp_mse
diff_autofcst_official = diff_autofcst_official.assign_coords(
    h=("lead_day", [2, 3, 4, 5, 6, 7, 8])
)
dm_result = diebold_mariano(diff_autofcst_official, "lead_day", "h")
dm_result